In [21]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
steps_per_sec = 24

In [57]:
data = np.load('preprocessed_data_sync/recording0.npy', allow_pickle=True)
inst = np.load('preprocessed_data_sync/instruments0.npy', allow_pickle=True)

In [64]:
def data_to_midi(seq, instrument_number):
    instrument=pretty_midi.Instrument(program=instrument_number,name=pretty_midi.program_to_instrument_name(instrument_number))
    
    # Keys are pitches, values are (velocity, start time)
    notes_on = dict()
    
    for tstep in range(seq.shape[0]):
        # Check if any notes are finished
        del_keys = set()
        for pitch in notes_on:
            if seq[tstep, num_notes + pitch] == 0:
                velocity = notes_on[pitch][0]
                start_time = notes_on[pitch][1]
                end_time = tstep/steps_per_sec
                note=pretty_midi.Note(velocity=velocity, pitch=pitch, start=start_time, end=end_time)
                instrument.notes.append(note)
                del_keys.add(pitch)
                
        for pitch in del_keys:
            del notes_on[pitch]
                    
        notes_being_played = np.argwhere(seq[tstep, :num_notes]).flatten()
        for pitch in notes_being_played:
            if pitch in notes_on:
                continue
                
            velocity = int(round(seq[tstep, 2*num_notes + pitch]*127))
            start_time = tstep/steps_per_sec
            notes_on[pitch] = (velocity, start_time)
    return instrument
    

In [65]:
recording = pretty_midi.PrettyMIDI()
for i,j in zip(data, inst):
    recording.instruments.append(data_to_midi(i,j))

In [66]:
recording.write("test_midi.mid")

In [67]:
file = open( "preprocessed_data_sync_fnames.p", "rb" )
res = pickle.load(file)
file.close()

In [68]:
res[0]

'Faure/2167_gr_f45m2.mid'